In [13]:
#pipeline imports
import pandas as pd
import nltk
import dodfminer

#Elementos do pipeline precisam herdar de TransformerMixin, sendo o ultimo (mais da direita) o Modelo que herda de BaseEstimator
from sklearn.base import TransformerMixin, BaseEstimator

#Pipeline precisa ser do sklearn.pipeline
from sklearn.pipeline import Pipeline


from sklearn_crfsuite import CRF


class FeatureExtractor_Tokenizer(TransformerMixin):
  def __init__(self):
    # print(">>>> init() Transformer called.\n")
    pass

  def get_features(self, sentence):
    sent_features = []
    for i in range(len(sentence)):
      word_feat = {
        # Palavra atual
        'word': sentence[i].lower(),
        'capital_letter': sentence[i][0].isupper(),
        'all_capital': sentence[i].isupper(),
        'isdigit': sentence[i].isdigit(),
        # Uma palavra antes
        'word_before': '' if i == 0 else sentence[i-1].lower(),
        'word_before_isdigit': '' if i == 0 else sentence[i-1].isdigit(),
        'word_before_isupper': '' if i == 0 else sentence[i-1].isupper(),
        'word_before_istitle': '' if i == 0 else sentence[i-1].istitle(),
        # Uma palavra depois
        'word_after': '' if i+1 >= len(sentence) else sentence[i+1].lower(),
        'word_after_isdigit': '' if i+1 >= len(sentence) else sentence[i+1].isdigit(),
        'word_after_isupper': '' if i+1 >= len(sentence) else sentence[i+1].isupper(),
        'word_after_istitle': '' if i+1 >= len(sentence) else sentence[i+1].istitle(),

        'BOS': i == 0,
        'EOS': i == len(sentence)-1
      }
      sent_features.append(word_feat)
    return sent_features

  def tokenize(self, sentence):
    text = nltk.word_tokenize(sentence)
    return text

  def fit(self, X, y = None):
    # print(">>>> fit() Transformer called.\n")
    return self

  def transform(self, X, y = None):
    # print(">>>> transform() Transformer called.\n")
    transformed = []
    for x in X:
      text = self.tokenize(x)
      features = self.get_features(text)
      transformed.append(features)
    return transformed


class FeatureExtractor(TransformerMixin):
  def __init__(self):
    # print(">>>> init() Transformer called.\n")
    pass

  def get_features(self, sentence):
    sent_features = []
    for i in range(len(sentence)):
      word_feat = {
        # Palavra atual
        'word': sentence[i].lower(),
        'capital_letter': sentence[i][0].isupper(),
        'all_capital': sentence[i].isupper(),
        'isdigit': sentence[i].isdigit(),
        # Uma palavra antes
        'word_before': '' if i == 0 else sentence[i-1].lower(),
        'word_before_isdigit': '' if i == 0 else sentence[i-1].isdigit(),
        'word_before_isupper': '' if i == 0 else sentence[i-1].isupper(),
        'word_before_istitle': '' if i == 0 else sentence[i-1].istitle(),
        # Uma palavra depois
        'word_after': '' if i+1 >= len(sentence) else sentence[i+1].lower(),
        'word_after_isdigit': '' if i+1 >= len(sentence) else sentence[i+1].isdigit(),
        'word_after_isupper': '' if i+1 >= len(sentence) else sentence[i+1].isupper(),
        'word_after_istitle': '' if i+1 >= len(sentence) else sentence[i+1].istitle(),

        'BOS': i == 0,
        'EOS': i == len(sentence)-1
      }
      sent_features.append(word_feat)
    return sent_features

  def fit(self, X, y = None):
    # print(">>>> fit() Transformer called.\n")
    return self

  def transform(self, X, y = None):
    # print(">>>> transform() Transformer called.\n")
    transformed = []
    for x in X:
      features = self.get_features(x)
      transformed.append(features)
    return transformed


class Processing(TransformerMixin):
  def __init__(self):
    # print(">>>> init() Transformer called.\n")
    pass

  def tokenize(self, sentence):
    text = nltk.word_tokenize(sentence)
    return text

  def fit(self, X, y = None):
    # print(">>>> fit() Transformer called.\n")
    return self

  def transform(self, X, y = None):
    # print(">>>> transform() Transformer called.\n")
    transformed = []
    for x in X:
      tokens = self.tokenize(x)
      transformed.append(tokens)
    return transformed


class Model(BaseEstimator):

    def __init__(self):
        # print(">>>> init() Estimator called.\n")
        self.crf = CRF(
          algorithm = 'lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=True
        )

    def transform(self, X, y = None):
      return X

    # This method will not be used in the default pipeline
    def fit(self, X, y):
        # print(">>>> fit() Estimator called.\n")
        # print("Training...\n")
        self.crf.fit(X, y)
        # print("Done!\n")
        return self

    def predict(self, x):
        # print(">>>> predict() Estimator called.\n")
        pred = self.crf.predict(x)
        return pred

In [14]:
# pega o json
# extrai os atos (segmentação do json)
# pega esses atos e joga no pipeline (default do dodfminer ou passado pelo usuario como parametro)
# verificar se pode montar o dataframe



#   Obrigatoriamente deve se ter um elemento com "chave" = "pre-processing" que é responsável pelo pre-processamento e tokenização, e esse processo deve ser chamado pelo método Pipeline['pre-processing'].transform(X) 
#   em que X são os dados de entrada do pipeline em formato de lista (array)

#   O modelo que herda do BaseEstimator deve ter sua saída em formato IOB. 

#   Sem essas duas condições não é possível montar o dataFrame com as entidades extraídas.

#   Caso não cumpra alguma das condições, as predições do modelo serão exibidas em formato DataFrame de forma desestruturada.

pipeline_CRF_custom = Pipeline([('pre-processing', Processing()), ('feature-extraction', FeatureExtractor()), ('model', Model())])

In [15]:
pipeline_CRF_custom['pre-processing'].transform(["oi eu sou o bruno"])

[['oi', 'eu', 'sou', 'o', 'bruno']]

In [16]:
pipeline_CRF_custom['pre-processing'].transform(["oi eu sou o bruno", "eu sou o andrei"])

[['oi', 'eu', 'sou', 'o', 'bruno'], ['eu', 'sou', 'o', 'andrei']]

In [17]:
# Usuario treinando o modelo


df_aditamento_contratual = pd.read_csv('https://raw.githubusercontent.com/brunoedcf/data_crf_training/main/aditamento_contratual.csv')

X = df_aditamento_contratual['treated_text'].to_list()
y = df_aditamento_contratual['IOB'].to_list()
y = [iob.split() for iob in y]

pipeline_CRF_custom.fit(X, y)

Pipeline(steps=[('pre-processing',
                 <__main__.Processing object at 0x7fb6fae54850>),
                ('feature-extraction',
                 <__main__.FeatureExtractor object at 0x7fb6fae54650>),
                ('model', Model())])

In [18]:
from dodfminer.extract.polished.core import ActsExtractor

res = ActsExtractor.get_act_obj("aditamento", "./jsons/DODF 151 11-08-2022.json", pipeline = pipeline_CRF_custom)

Foram encontrados 5 atos de aditamento


In [19]:
res.df

,numero_dodf,titulo,text,NUM_ADITIVO,PROCESSO,CONTRATANTE,OBJ_ADITIVO,DATA_ESCRITO,CODIGO_SIGGO,NUM_AJUSTE
0,151,EXTRATO DO 16º TERMO ADITIVO AO CONTRATO CCER ...,EXTRATO DO 16º TERMO ADITIVO AO CONTRATO CCER ...,16º TERMO ADITIVO,"[00080-00193050/2020-95, 06/04/2022]","[SEEDF, SEEDF]",a inclusão dos Dados da Unidade Consumidora so...,NaN,NaN,NaN
1,151,EXTRATO DO 16º TERMO ADITIVO AO CONTRATO CUSD ...,EXTRATO DO 16º TERMO ADITIVO AO CONTRATO CUSD ...,16º TERMO ADITIVO,00080-00193050/2020-95,"[SEEDF, SEEDF]",a inclusão dos Dados da Unidade Consumidora so...,22/02/2022,NaN,NaN
2,151,EXTRATO DO 9° TERMO ADITIVO AO CONTRATO Nº 06/...,EXTRATO DO 9° TERMO ADITIVO AO CONTRATO Nº 06/...,9° TERMO ADITIVO,"[0417-000806/2013, 05/08/2022]",Secretaria de Estado de Justiça e Cidadania,A alteração contratual com vistas a modificar ...,NaN,29365,NaN
3,151,EXTRATO DO TERCEIRO TERMO ADITIVO DE PRORROGAÇ...,EXTRATO DO TERCEIRO TERMO ADITIVO DE PRORROGAÇ...,TERCEIRO TERMO ADITIVO,NaN,SECRETARIA DE ESTADO DE OBRAS E INFRAESTRUTURA...,"Sob o amparo do artigo 57 , §1º , incisos IV e...",08 de agosto de 2022,NaN,"[014/2021, 00110-00002885/2021-31, 00110-00003..."
4,151,EXTRATO DO PRIMEIRO TERMO ADITIVO AO CONTRATO ...,EXTRATO DO PRIMEIRO TERMO ADITIVO AO CONTRATO ...,PRIMEIRO TERMO ADITIVO,"[04012-00000195/2021-15, 08/08/2022]",SECRETARIA DE ESTADO DE TRABALHO DO DISTRITO F...,2.1 . O presente Termo Aditivo tem por objeto ...,NaN,NaN,015/2021


In [20]:
res = ActsExtractor.get_act_obj("aditamento", "./jsons/DODF 151 11-08-2022.json")
res.df

Foram encontrados 5 atos de aditamento


,numero_dodf,titulo,text,NUM_ADITIVO,NUM_AJUSTE,CONTRATANTE,OBJ_ADITIVO,PROCESSO,DATA_ESCRITO,CODIGO_SIGGO
0,151,EXTRATO DO 16º TERMO ADITIVO AO CONTRATO CCER ...,EXTRATO DO 16º TERMO ADITIVO AO CONTRATO CCER ...,16º TERMO ADITIVO,0714/2017,"[SEEDF, SEEDF]",a inclusão dos Dados da Unidade Consumidora so...,06/04/2022,NaN,NaN
1,151,EXTRATO DO 16º TERMO ADITIVO AO CONTRATO CUSD ...,EXTRATO DO 16º TERMO ADITIVO AO CONTRATO CUSD ...,16º TERMO ADITIVO,0714/2017,"[SEEDF, SEEDF]",a inclusão dos Dados da Unidade Consumidora so...,NaN,22/02/2022,NaN
2,151,EXTRATO DO 9° TERMO ADITIVO AO CONTRATO Nº 06/...,EXTRATO DO 9° TERMO ADITIVO AO CONTRATO Nº 06/...,9° TERMO ADITIVO,NaN,Secretaria de Estado de Justiça e Cidadania,A alteração contratual com vistas a modificar ...,"[0417-000806/2013, 05/08/2022]",NaN,29365
3,151,EXTRATO DO TERCEIRO TERMO ADITIVO DE PRORROGAÇ...,EXTRATO DO TERCEIRO TERMO ADITIVO DE PRORROGAÇ...,TERCEIRO TERMO ADITIVO,"[014/2021, 00110-00001782/2020-72, 00110-00002...",SECRETARIA DE ESTADO DE OBRAS E INFRAESTRUTURA...,"Sob o amparo do artigo 57 , §1º , incisos IV e...",NaN,08 de agosto de 2022,NaN
4,151,EXTRATO DO PRIMEIRO TERMO ADITIVO AO CONTRATO ...,EXTRATO DO PRIMEIRO TERMO ADITIVO AO CONTRATO ...,PRIMEIRO TERMO ADITIVO,015/2021,SECRETARIA DE ESTADO DE TRABALHO DO DISTRITO F...,2.1 . O presente Termo Aditivo tem por objeto ...,"[04012-00000195/2021-15, 08/08/2022]",NaN,NaN


In [21]:
pipeline_CRF_custom_incomplete = Pipeline([('feature-extraction', FeatureExtractor_Tokenizer()), ('model', Model())])

In [22]:
df_aditamento_contratual = pd.read_csv('https://raw.githubusercontent.com/brunoedcf/data_crf_training/main/aditamento_contratual.csv')

X = df_aditamento_contratual['treated_text'].to_list()
y = df_aditamento_contratual['IOB'].to_list()
y = [iob.split() for iob in y]

pipeline_CRF_custom_incomplete.fit(X, y)

Pipeline(steps=[('feature-extraction',
                 <__main__.FeatureExtractor_Tokenizer object at 0x7fb6fbffca50>),
                ('model', Model())])

In [23]:
res = ActsExtractor.get_act_obj("aditamento", "./jsons/DODF 151 11-08-2022.json", pipeline = pipeline_CRF_custom_incomplete)
res.df

Foram encontrados 5 atos de aditamento


,0,1,2,3,4,5,6,7,8,9,...,526,527,528,529,530,531,532,533,534,535
0,O,O,B-NUM_ADITIVO,I-NUM_ADITIVO,I-NUM_ADITIVO,O,O,O,O,O,...,None,None,None,None,None,None,None,None,None,None
1,O,O,B-NUM_ADITIVO,I-NUM_ADITIVO,I-NUM_ADITIVO,O,O,O,O,O,...,None,None,None,None,None,None,None,None,None,None
2,O,O,B-NUM_ADITIVO,I-NUM_ADITIVO,I-NUM_ADITIVO,O,O,O,O,O,...,None,None,None,None,None,None,None,None,None,None
3,O,O,B-NUM_ADITIVO,I-NUM_ADITIVO,I-NUM_ADITIVO,O,O,O,O,O,...,O,O,O,O,O,O,O,O,O,O
4,O,O,B-NUM_ADITIVO,I-NUM_ADITIVO,I-NUM_ADITIVO,O,O,O,O,O,...,None,None,None,None,None,None,None,None,None,None


In [24]:
res.predicted

[['O',
  'O',
  'B-NUM_ADITIVO',
  'I-NUM_ADITIVO',
  'I-NUM_ADITIVO',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-PROCESSO',
  'O',
  'O',
  'O',
  'B-CONTRATANTE',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'I-OBJ_ADITIVO',
  'O',
  'O',
  'O',
  'B-PROCESSO',
  'O',
  